In [10]:
import numpy as np
import torch
from torch import nn
import onnx
import onnxsim
import onnxslim
import onnxruntime as ort

# 定义model

In [11]:
class Model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(10, 5)
        self.act1 = nn.ReLU(inplace=True)
        self.dropout = nn.Dropout(0.1)
        self.fc2 = nn.Linear(5, 2)

        self.register_buffer("layers", torch.tensor(2))

        # 这样保存的权重没法导出为onnx
        self.attr = {0: "dog", 1: "cat"}

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.fc1(x)
        x = self.act1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# export example

In [12]:
model = Model()

In [13]:
x = torch.ones(1, 10)
onnx_path = r"./models/model_with_attrs.onnx"

In [14]:
torch.onnx.export(
    model,  # 保存的模型
    x,  # 模型输入
    onnx_path,  # 模型保存 (can be a file or file-like object)
    opset_version=17,  # ONNX version 值必须等于_onnx_main_opset或在_onnx_stable_opsets之内。具体可在torch/onnx/symbolic_helper.py中找到
    input_names=["input"],  # 按顺序分配给onnx图的输入节点的名称列表
    output_names=["output"],  # 按顺序分配给onnx图的输出节点的名称列表
)

# add extra metadata

In [16]:
model = onnx.load(onnx_path)

In [17]:
d = {"layers": 2, "classes": {0: "dog", 1: "cat"}}
for k, v in d.items():
    meta = model.metadata_props.add()
    meta.key, meta.value = k, str(v)

# simplify model

In [18]:
model_ = onnxslim.slim(model)

In [19]:
onnx.save(model_, onnx_path)

# onnxruntime

In [20]:
so = ort.SessionOptions()
so.log_severity_level = 3
ort_model = ort.InferenceSession(
    onnx_path,
    sess_options=so,
    providers=["CUDAExecutionProvider", "CPUExecutionProvider"],
)

In [21]:
res = ort_model.run(
    None, {ort_model.get_inputs()[0].name: np.ones((1, 10), dtype=np.float32)}
)

In [22]:
res[0].shape

(1, 2)